<a href="https://colab.research.google.com/github/donghui-0126/machine-learning/blob/main/%ED%95%B8%EC%A6%88%EC%98%A8%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/2%EC%9E%A5%20%EB%94%A5%EB%9F%AC%EB%8B%9D/%5B16%EC%9E%A5%5D_RNN%EA%B3%BC_%EC%96%B4%ED%85%90%EC%85%98%EC%9D%84_%EC%82%AC%EC%9A%A9%ED%95%9C_%EC%9E%90%EC%97%B0%EC%96%B4_%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- conding:utf-8 -*-

# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 텐서플로 ≥2.0 필수
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("감지된 GPU가 없습니다. GPU가 없으면 CNN은 매우 느릴 수 있습니다.")
    

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)
tf.random.set_seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

이번 장에서는 자연어를 읽고 쓰는 AI를 만드는 법을 알아볼 것이다. 

자연어 문제를 위해 많이 사용하는 방법은 순환 신경망이다. 따라서 RNN을 계속해서 살펴본다. 문장에서 다음글자를 예측하도록 훈련하는 **문자 단위 RNN(character RNN)** 부터 시작한다. 새로운 텍스트를 생성하고 그 과정에서 매우 긴 시퀀스를 가진 텐서플로 데이터셋을 만드는 방법을 알아보자. 먼저 **상태가 없는 RNN(stateless RNN)** 을 사용하고 다음에 **상태가 있는 RNN(stateful RNN)** 을 구축할 것이다. (상태가 없는 RNN은 각 반복에서 무작위하게 택한 텍스트의 일부분으로 학습하고, 나머지 텍스트에서 어떤 정보도 사용하지 않는다. 상태가 있는 RNN은 훈련 반복 사이에 은닉 상태를 유지하고 중지된 곳에서 이어서 상태를 반영한다. 그래서 더 긴패턴을 학습할 수 있다). 그 다음에는 감성분석을 수행하는 RNN을 구축하고 이번에는 문자가 아니라 단어의 시퀀스로 문장을 다룰것이다. 그리고 신경망 기계번역(NMT)을 수행할 수 있는 인코더-디코더 구조를 만들기 위해 RNN을 사용하는 방법을 알아보자. 도구로는 텐서플로 애드온 프로젝트에서 제공하는 seq2seq API를 사용한다.

16.4절에서는 어텐션 매커니즘을 알아보자. 이름에서 알 수 있듯이 이는 각 타임 스텝에서 모델이 집중해야 할 입력 부분을 선택하도록 학습하는 신경망 구성요소이다. 먼저 어텐션을 사용하여 RNN 기반의 인코더-디코더 구조의 성능을 높이는 방법을 알아본 뒤, RNN을 모두 제거하고 어텐션만 사용해 매우좋은 성능을 내는 **트랜스포머** 라는 구조도 살펴보자.  마지막으로 NLP분야에서 가장 중요한 발전을 살펴보자. 트랜스포머를 시반으로 한 GPT-2와 BERT같은 매우 강력한 언어 모델도 포함한다.

# char-RNN을 사용해 셰익스피어 문체 생성하기

In [2]:
shakespeare_url = "https://homl.info/shakespeare"  # 단죽 url
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1115394/1115394 [==============================] - 0s 0us/step


In [3]:
# 모든 글자를 정수로 인코딩해준다. 
# 간단하게 Tokenizer 클래스를 사용한다.

# char_level 을 True로 지정해서 단어 수준 인코딩 대신 글자 수준 인코딩을 만든다. 이 클래스는 기본적으로 텍스트를 소문자로 바꾼다.
# 이제 문장을 글자 ID로 인코딩 하거나 반대로 디코딩 할 수 있다. 
# 이를 통해텍스트에 있는 고유 글자 개수와 전체 글자 개수를 알 수 있다.
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [4]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [5]:
tokenizer.sequences_to_texts([[20,6,9,8,3]])

['f i r s t']

In [6]:
# 고유 글자 개수
max_id = len(tokenizer.word_index)
max_id

39

In [7]:
# 전체 글자 개수
dataset_size = tokenizer.document_count 
dataset_size

1115394

In [8]:
# 전체 텍스트를 인코딩해서 각 글자를 ID로 나타내 보자.
# 1~ 39인 값을 0~38로 바꾸기
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [9]:
encoded.shape

(1115394,)

## 순차 데이터셋을 나누는 방법
훈련셋, 테스트셋, 검증셋을 중복되지 않도록 만드는 것은 중요하다.

보통 시계열을 다룰 때는 시간에 따라 나눈다. 다른 차원으로 나눌 수 있지만, train set 과 test set 사이의 상관관계가 있다면 오차를 낙관적으로 측정해서 모델이 유용하지 않을 것이다. 

시계열 데이터가 시간에 따라서 반복되는 **변하지 않는 상태** 라면 시계열을 안정적으로 분석할 수 있다. 하지만 그렇지 않은 시계열도 있을 것이다. 

시계열이 안정적인지 확인할 방법은 시간에 따라 검증 세트에 대한 모델의 오차를 그리는 것이다. 모델이 검증 세트 마지막 보다 첫 부분에서 성능이 더 좋다면 이 시계열이 충분히 안정되지 않을 것일 수 있다. 이런 경우에는 **더 짧은 사건 간격으로** 모델을 훈련하는 게 좋다.

In [10]:
# 셰익스피어 텍스트의 처음 90%를 trainset으로 사용하고 나머지는 validset과 testset로 사용한다. 
# 이 set에서 한 번에 한 글자씩 반환하는 tf.data.Dataset 객체를 만든다.
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
train_size

1003854

## 순차 데이터를 윈도 여러개로 자르기

https://teddylee777.github.io/tensorflow/dataset-batch-window  이해가 잘 안되서 찾아본 블로그이다. 

trainset은 백만개 이상의 글자로 이루어진 시퀀스 하나이다. 여기에 신경망을 직접 훈련시킬수는 없다. 이 RNN은 백만 개의 층이 있는 심층 신경망과 비슷하고 (매우 긴) 샘플 하나로 훈련하는 셈이 된다. 대신 데이터셋의 window() 메서드를 사용해 이 긴 시퀀스를 작은 많은 텍스트 윈도로 변환한다. 이 데이터셋의 각 샘플은 전체 텍스트에서 매우 짧은 부분 문자열이다. RNN은 이 부분문자열 길이만큼만 역전파를 위해 펼쳐진다. 이를 **TBPTT** 라고 한다. window() 메서드를 호출하여 짧은 텍스트 윈도를 갖는 데이터셋을 만들어보자.

In [11]:
# n_steps를 튜닝할 수 있다.
# 짧은 입력 시퀀스에서 RNN을 훈련하는 것은 쉽지만 당연히 이 RNN은 n_steps 보다 긴 패턴을 학습할 수 없다.
n_steps = 100
window_length = n_steps + 1 # target = 한글자앞의 input
# window_depth: 윈도우 크기 
# shift: 원래는 윈도우크기가 default값이여서 윈도우가 겹치지 않음. shift=1 이면 최대크기의 trainset 생성
# drop_remainder: True로 지정하면 같은 크기의 윈도우를 생성함. False면 윈도우크기가 1씩 줄어든다. 
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

window()메서드를 사용하면 각각 하나의 데이터셋으로 표현되는 윈도를 포함하는 데이터셋을 만든다. 리스트의 리스트와 비슷한 nested dataset이다. 이런 구조는 데이터셋 메서드를 호출하여 각 윈도를 변환할 때 유용하다. 하지만 모델은 데이터셋이 아니라 tensor를 기대하기 때문에 훈련에 데이터셋을 바로 사용할 수 없다. 따라서 flat_map()을 통해서 nested dataset을 flat dataset으로 변환해주어야한다. 

flat_map()메서드는 중첩 dataset을 평평하게 만들기 전에 각 데이터셋에 적용할 변환 함수를 매개변수로 받을 수 있다. 예를 들어 lambda ds: ds.batch(2) 함수를 flat_map()에 전달하면 텐서 2개를 가진 테이터셋으로 변환해준다.

In [12]:
# 윈도마다 batch(window_length)를 호출한다. 이 길이는 윈도 길이와 같기 때문에 텐서 하나를 담은 데이터 셋을 얻는다.
# 이 데이터셋은 연속된 101글자 길이의 윈도를 담는다.
dataset = dataset.flat_map(lambda window: window.batch(window_length))

경사하강법은 훈련 세트 샘플이 동일 독립 분포 일 떄 가장 잘 작동하기 때문에 이 윈도를 섞어야한다. 그다음 윈도를 배치로 만들고 입력과 타깃으로 분리하겠다.

In [13]:
np.random.seed(42)
tf.random.set_seed(42)

batch_size= 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [14]:
# 범주형 입력특성이므로 원핫인코딩을 진행한다. (고유한 글자 수가 적기 때문에 임베딩 대신 원핫 인코딩을 사용함).
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

마지막으로 프리페칭을 추가한다.

In [15]:
dataset = dataset.prefetch(1)

In [16]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


13장에 나온 데이터 전처리와 적재 파트의 공부를 건너 뛰어서 살짝 잘 모르겠는 부분이 있다. 이제 필요성을 느꼈으니 공부를 해야겠다.

## Char-RNN 모델 훈련

In [17]:
np.random.seed(42)
tf.random.set_seed(42)

# GRU 클래스는 다음 매개변수에서 기본값을 사용할 때에만 GPU를 사용합니다: 
# activation, recurrent_activation, recurrent_dropout, unroll, use_bias reset_after. 이 때문에 (책과는 달리) recurrent_dropout=0.2를 주석 처리했습니다.

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                   # dropout = 0.2, recurrent_dropout = 0.2),
                   dropout = 0.2),
    keras.layers.GRU(128, return_sequences=True,
                   # dropout = 0.2, recurrent_dropout = 0.2),
                   dropout = 0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])


# 손실함수는 https://didu-story.tistory.com/27 참조
# 라벨이 정수인코딩 된 경우에 사용함
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=10)

Epoch 1/10
31368/31368 [==============================] - 420s 13ms/step - loss: 1.6238
Epoch 2/10
31368/31368 [==============================] - 392s 12ms/step - loss: 1.5397
Epoch 3/10
31368/31368 [==============================] - 396s 13ms/step - loss: 1.5191
Epoch 4/10
31368/31368 [==============================] - 389s 12ms/step - loss: 1.5076
Epoch 5/10
31368/31368 [==============================] - 393s 12ms/step - loss: 1.5000
Epoch 6/10
31368/31368 [==============================] - 411s 13ms/step - loss: 1.4943
Epoch 7/10
31368/31368 [==============================] - 397s 13ms/step - loss: 1.4906
Epoch 8/10
31368/31368 [==============================] - 394s 13ms/step - loss: 1.4876
Epoch 9/10
31368/31368 [==============================] - 390s 12ms/step - loss: 1.4854
Epoch 10/10
31368/31368 [==============================] - 397s 13ms/step - loss: 1.4828


In [22]:
model.save("my_shakespeare_model.h5")

In [23]:
model = keras.models.load_model("my_shakespeare_model.h5")

## Char-RNN 모델 사용하기.
이제 셰익스피어가 쓴 텍스트에서 다음 글자를 예측해주는 모델을 얻었다. 이 모델에 새로운 텍스트를 주입하려면 앞에서와 같이 먼저 전처리를 해야한다.

In [18]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [19]:
X_new = preprocess(["How are yo"])
#Y_pred = model.predict_classes(X_new)
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'u'

In [28]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        2, 0, 0, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 0, 0, 0, 0]])

## 가짜 셰익스피어 텍스트 생성하기 
Char-RNN 모델을 사용해 새로운 텍스트를 생성하려면 먼저 초기 텍스트를 주입하고 모델이 가장 가능성 있는 다음 글자를 예측한다. 이 글자는 텍스트 끝에 추가하고 늘어난 텍스트를 모델에 전달하여 다음 글자를 예측하는 방식이다. 실제로는 이렇게 하면 같은 단어가 계속 반복될 수 있다. 대신 tf.random.categorical()함수를 이용해 모델이 추정한 확률을 기반으로 다음 글자를 무작위로 선택할 수 있다.

categorical() 함수는 클래스의 로그 확률(로직)을 전달하면 랜덤하게 클래스 인덱스를 샘플링한다. 생성된 텍스트의 다양성을 더 많이제어하려면 temperature 라고 불리는 숫자로 로짓을 나눈다. 온도가 낮을수록 랜덤성이 줄고 온도가 높을 수록 랜덤성이 높아진다. 

In [35]:
# 위 방식을 사용해서 다음 글자를 선택하고 그 텍스트를 반환한다.
def next_char(text, temperature=1):

    # X_new 형태와 y_proba 형태 살펴보기
    
    
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [36]:
tf.random.set_seed(42)

next_char("How are yo", temperature=1)

'u'

In [37]:
# 위 함수를 반복 호출하고 텍스트뒤에 합쳐주는 함수
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [38]:
print(complete_text("t", temperature=1))

the off,
did he er this verona on a kind of yoursel


## 상태가 있는 RNN